# Data Modelling With Cassandra

## Import packages

In [94]:
import os
import glob
import csv
from cassandra.cluster import Cluster

## ETL Pipeline for Pre-Processing the Files

#### Creating list of filepaths to process original event csv data files

In [95]:
# Get event data path
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect all filepath   
file_path_list = glob.glob(os.path.join(filepath,'*'))

#### Define functions for pre-processing

In [96]:
def get_all_event_data(file_path_list):
    full_data_rows_list = [] 
    
    # for every filepath in the file path list 
    for f in file_path_list:

        # reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)

            # extracting each data row one by one and append it        
            for line in csvreader:
                full_data_rows_list.append(line) 
                
    return full_data_rows_list          

def create_event_datafile_new(full_data_row_list):
    # creating a smaller event data csv file called event_datafile_new.csv 
    # that will be used to insert data into the Apache Cassandra tables
    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_row_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

#### Create the new data set 'event_datafile_new.csv'

In [97]:
full_data_row_list=get_all_event_data(file_path_list)
create_event_datafile_new(full_data_row_list)

## Set Apache Cassandra Cluster

#### Creating a Cluster

In [98]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [99]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [100]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## <br>Create tables and queries to ask the following three questions of the data

### Query1: 
#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4<br>

#### Data Modeling

1. The combination of sessionId and itemInSession is unique, and we need results based on sessionId and itmeInSession, so these two columns could be the Composite Primary Key of the table.  
2. Since itemInSession could be skewed to some specific value, it's better to use sessionId as Priamary key so the data could be evenly distributed on the nodes
3. We need the artist name, song title and song length based on the sessionId and itemInSession
Based on the above requirements, we could design the data model as follows:
    
```
Create table sessions
    (
        sessionId int,
        itemInSession int,
        artist_name text,
        song text,
        song_length float,
        PRIMARY KEY (sessionId, itemInSession)  
    ) WITH CLUSTERING ORDER BY (itemInSession DESC)
```

#### Create the table sessions

In [102]:
query = "CREATE TABLE IF NOT EXISTS sessions"
query = query + "(sessionId int, itemInSession int, artist_name text, song text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### <br>Insert data into the table sessions

In [103]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "INSERT INTO sessions(sessionId, itemInSession,artist_name, song, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### <br>Run the test query

In [113]:
query = "select artist_name, song, song_length from sessions WHERE sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Faithless', song='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### <br>**Query 2**: 
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182<br>

### Data Modeling

1. Since we need results based on userid and sessionid, it could make good perfomance to use userid and sessionid as partition key 
to make sure the sessions belonging to the same user could be distributed to same node
2. Since the data needs to be sorted by itemInSession, the itemInSession should be the clustering key
3. Besides Primary Key, we need artist name, song title and user name for this query
Based on the above requirements, we could design the data model as follows:

```
Create table song_play_list
    (
        userid int,
        sessionid int,
        iteminsession int,
        firstname text,
        lastname text,      
        artist_name text,
        song text,     
        PRIMARY KEY((userid, sessionid), iteminsession)
     ) WITH CLUSTERING ORDER BY (iteminsession DESC)

```


#### Create the table song_playlist_session

In [105]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userid int, firstname text, lastname text, sessionid int, artist_name text, song text, iteminsession int, \
PRIMARY KEY((userid, sessionid), iteminsession)) WITH CLUSTERING ORDER BY (iteminsession DESC);" 
try:
    session.execute(query)
except Exception as e:
    print(e)

#### <br>Insert data into the table song_playlist_session

In [106]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        query = "INSERT INTO song_playlist_session(userid, firstname, lastname, sessionid, artist_name, song, iteminsession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
     
        session.execute(query, (int(line[10]), line[1], line[4], int(line[8]), line[0],line[9],int(line[3])))

#### <br>Run the test query

In [111]:
query = "select artist_name, song, firstname,lastname, iteminsession from song_playlist_session where userid=10 and sessionid=182 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print ( row.artist_name,row.song,row.firstname, row.lastname, row.iteminsession)
    print(row)

Row(artist_name='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)
Row(artist_name='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist_name='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist_name='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)


### <br>**Query 3**: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'<br>

### Data Modeling

1. Since we need the results based on song name, firstName and lastName might be not unique, we need to use song name and userid as Primary Key
2. Use the song name as partition key so the users data listened to the same song could be distributed on the same node, it will improve the query performance
3. we need firatName and lastName as well for this query
Based on the above requirements, we could design the data model as follows:

Create table users_playlist 
    (
        song text,
        user_id,
        firstname text,
        lastname text,  
        PRIMARY KEY (song, user_id)  
    ) 


#### <br>Create the table users_playlist

In [88]:
query = "CREATE TABLE IF NOT EXISTS users_playlist"
query = query + "(song text,  userid int, firstname text, lastname text,  PRIMARY KEY (song, userid)) WITH CLUSTERING ORDER BY (userid DESC);" 
try:
    session.execute(query)
except Exception as e:
    print(e)

#### <br>Insert data into the table users_playlist

In [89]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO users_playlist(song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### <br>Run the test query

In [90]:
query = "select userid, firstname, lastname from users_playlist where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.userid, row.firstname, row.lastname)

95 Sara Johnson
80 Tegan Levine
29 Jacqueline Lynch


## Drop tables

In [91]:
## Drop the table before closing out the sessions
query = "drop table sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table users_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Close the session and cluster connection¶

In [92]:
session.shutdown()
cluster.shutdown()